# <center> Allrecipes analysis - part 1: preparing data </center>

Aim of this project is to prepare data collected from the internet for further analysis. Data was collected with a web scraper available as the following file in the Github repository - allrecipes_recipe_scraper.py. All data collected with the scraper was saved as a JSON file (data.json) and is available in the repository. Additionally, the list of herbs was compiled from the Encyclopedia Britannica and other online sources. The scraped data consists of 1000 records representing individual recipes from www.allrecipes.com website and 14 columns describing their characteristics. Below you can see the scope of the collected data:

* name: recipe name
* prep: preparation time
* cook: cooking time
* additional: additional time
* total: total time to prepare the recipe
* servings: number of dish servings received from recipe
* yield: number of dish servings in an appropriate measurement unit
* 5 stars: number of 5-star reviews received by the recipe
* 4 stars: number of 4-star reviews received by the recipe
* 3 stars: number of 3-star reviews received by the recipe
* 2 stars: number of 2-star reviews received by the recipe
* 1 stars: number of 1-star reviews received by the recipe
* nutrition: list of nutrients with assigned values
* ingredients: list of all ingredients used in the recipe

Number of analysed recipes was limited to 1000 to limit computational load but I believe it could be reproduced with more records if needed.

In the following steps we will be preparing data for further exploratory analysis. You can think about it as of a data cleaning part of the analysis project.

The initial step of our project is to import all necessary modules that we are going to use throughout the data cleaning process.

In [1]:
import pandas as pd
import numpy as np
import random

Once we’ve imported all necessary modules, we can read the data from data.json file and save it as pandas Dataframe in the allrecipes_df variable.

Additionally, since we have quite a bit of data to take a look at, we will set the display options of pandas to print all the columns.

In [2]:
allrecipes_df = pd.read_json('data.json', orient = 'index')

pd.set_option('display.max_columns', None)

Now that we have the data available in allrecipes_df variable we can perform some basic data exploration to get a grasp of what are we dealing with. In the following few lines of code, we are going to check all available columns of the Dataframe and data types associated with each column. Additionally, we will print out first few rows of the data set.

In [3]:
print(allrecipes_df.columns)

Index(['name', 'prep:', 'cook:', 'additional:', 'total:', 'Servings:',
       'Yield:', '5 stars', '4 stars', '3 stars', '2 stars', '1 stars',
       'nutrition', 'ingredients'],
      dtype='object')


In [4]:
allrecipes_df.dtypes

name            object
prep:           object
cook:           object
additional:     object
total:          object
Servings:        int64
Yield:          object
5 stars        float64
4 stars        float64
3 stars        float64
2 stars        float64
1 stars        float64
nutrition       object
ingredients     object
dtype: object

In [5]:
allrecipes_df.head()

,name,prep:,cook:,additional:,total:,Servings:,Yield:,5 stars,4 stars,3 stars,2 stars,1 stars,nutrition,ingredients
1,Juicy Roasted Chicken,10 mins,1 hr 15 mins,15 mins,1 hr 40 mins,6,6 servings,3179.0,538.0,147.0,47.0,42.0,423 calories; protein 30.9g; carbohydrates 1.2...,"1 (3 pound) whole chicken, giblets removed,sal..."
2,Microwave Corn on the Cob,None,5 mins,None,5 mins,1,1 serving,382.0,102.0,29.0,5.0,6.0,123 calories; protein 4.6g; carbohydrates 27.2...,"1 ear corn, husked and cleaned"
3,French Toast I,5 mins,15 mins,None,20 mins,3,6 slices french toast,1337.0,473.0,87.0,31.0,29.0,240 calories; protein 10.6g; carbohydrates 33....,"6 thick slices bread,2 eggs,⅔ cup milk,¼ teas..."
4,The Best Banana Pudding,25 mins,None,None,25 mins,20,20 servings,830.0,117.0,30.0,15.0,15.0,329 calories; protein 4.2g; carbohydrates 56.9...,1 (5 ounce) package instant vanilla pudding mi...
5,Simple Macaroni and Cheese,10 mins,20 mins,None,30 mins,4,4 servings,545.0,188.0,51.0,26.0,31.0,630 calories; protein 26.5g; carbohydrates 55g...,"1 (8 ounce) box elbow macaroni,¼ cup butter,¼ ..."


We will also check some statistical measures with the Dataframe describe() method. It will calculate count, min, max, mean, standard deviation and 25th/75th percentiles in all columns that consists of numerical values. Columns with non-numerical values are automatically omitted.

In [6]:
allrecipes_df.describe()

,Servings:,5 stars,4 stars,3 stars,2 stars,1 stars
count,1000.000000,980.000000,980.000000,980.000000,980.000000,980.000000
mean,9.782000,859.734694,202.681633,52.773469,19.970408,17.451020
std,11.847736,1467.753844,282.735392,79.691308,32.956996,35.097481
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,133.750000,35.000000,8.000000,3.000000,2.000000
50%,6.000000,381.500000,102.500000,26.000000,9.000000,7.000000
75%,12.000000,934.750000,242.250000,63.000000,23.000000,18.000000
max,192.000000,14902.000000,2581.000000,810.000000,362.000000,606.000000


The important part of any data analysis project is to verify whether there are any missing values in the dataset. We are going to check how many records (individual recipes) have at least one characteristic (value in a specific column) missing. Next, we will check how many missing values occur in each column of the dataset.

In [7]:
rows_with_missing_values = allrecipes_df.isnull().any(axis = 1).sum() # in the whole dataset
print(f'Number of rows with missing values: {rows_with_missing_values}')

Number of rows with missing values: 769


In [8]:
allrecipes_df.isnull().sum(axis = 0) # count per column

name             0
prep:           52
cook:          183
additional:    717
total:          49
Servings:        0
Yield:           0
5 stars         20
4 stars         20
3 stars         20
2 stars         20
1 stars         20
nutrition        1
ingredients      0
dtype: int64

There is over 70% of missing values in the ‘additional’ column of our Dataframe. In some circumstances it would be enough to drop this column as useless. In our case, however, ‘additional’ column will play a role in the future and therefore we are going to leave it for now.

Now that we have learned a little bit about the data, we can start the cleaning process. We are going to start with fixing column names formatting. In this step we will remove all whitespaces and colons from the end of a column names and put it all in lower case.

In [9]:
allrecipes_df.rename(columns = lambda x: x.lower().strip().strip(':'), inplace = True)
print(allrecipes_df.columns)

Index(['name', 'prep', 'cook', 'additional', 'total', 'servings', 'yield',
       '5 stars', '4 stars', '3 stars', '2 stars', '1 stars', 'nutrition',
       'ingredients'],
      dtype='object')


The column names are nicely formatted now. This should help us in the future when we try to refer to any particular piece of data. In the following steps we will create some new columns and modify the existing ones. This process should enrich our dataset and help us ultimately get to some meaningful findings about the recipes. We can start with creating column ‘recipe_score’ containing information regarding recipe’s overall scoring.

In [10]:
allrecipes_df['recipe_score'] = (allrecipes_df['1 stars'] * 1
                                + allrecipes_df['2 stars'] * 2
                                + allrecipes_df['3 stars'] * 3
                                + allrecipes_df['4 stars'] * 4
                                + allrecipes_df['5 stars'] * 5) / (allrecipes_df['1 stars'] 
                                                                   + allrecipes_df['2 stars'] 
                                                                   + allrecipes_df['3 stars'] 
                                                                   + allrecipes_df['4 stars'] 
                                                                   + allrecipes_df['5 stars'])

Next, we should create a column ‘number_of_reviews’ containing number of total reviews that recipe got. Without this information, it would be difficult to asses whether the overall score is representative or not in the future.

In [11]:
allrecipes_df['number_of_reviews'] = (allrecipes_df['1 stars']
                                     + allrecipes_df['2 stars']
                                     + allrecipes_df['3 stars']
                                     + allrecipes_df['4 stars']
                                     + allrecipes_df['5 stars'])

We know that ‘nutrients' column contains information about the nutritional components of a recipe. Those information are currently stored as one long string. In the current format ‘nutrients’ column doesn’t allow for any quantitative analysis. If we want to extract meaningful information from ‘nutrients’ column, firstly we need to see how exactly are the information stored now and whether there are any data points missing. If missing values occur, we might need to incorporate exception handling in the process of extracting information about nutrients.

In [12]:
allrecipes_df['nutrition'].head(10)

1     423 calories; protein 30.9g; carbohydrates 1.2...
2     123 calories; protein 4.6g; carbohydrates 27.2...
3     240 calories; protein 10.6g; carbohydrates 33....
4     329 calories; protein 4.2g; carbohydrates 56.9...
5     630 calories; protein 26.5g; carbohydrates 55g...
6     507 calories; protein 33.1g; carbohydrates 8.7...
7     170 calories; protein 4.8g; carbohydrates 28.1...
8     247 calories; protein 6.8g; carbohydrates 33.5...
9     333 calories; protein 9.8g; carbohydrates 30.8...
10    252 calories; protein 4.5g; carbohydrates 29.7...
Name: nutrition, dtype: object

In [13]:
allrecipes_df[allrecipes_df['nutrition'].isnull()]   

,name,prep,cook,additional,total,servings,yield,5 stars,4 stars,3 stars,2 stars,1 stars,nutrition,ingredients,recipe_score,number_of_reviews
368,Campbell's® Tuna Noodle Casserole,10 mins,35 mins,None,45 mins,8,8 servings,252.0,147.0,47.0,10.0,3.0,None,2 (10.75 ounce) cans Campbell's® Condensed Cr...,4.383442,459.0


It turns out that there is only one recipe for which nutrients information are not available. Now that we have a better overview of the situation we can start the process of separating information about individual nutrients into respective columns. First, we are going to create an empty column per each nutrient.

In [14]:
allrecipes_df['calories'] = np.nan
allrecipes_df['protein'] = np.nan
allrecipes_df['carbohydrates'] = np.nan
allrecipes_df['fat'] = np.nan
allrecipes_df['cholesterol'] = np.nan
allrecipes_df['sodium'] = np.nan

In the following part we will separate the information from the ‘nutrients’ column. To achieve this we will perform the following steps while iterating over each row in the column:
1. Split the string into list of individual words
2. Strip all whitespaces and semicolons from words in the list
3. For each word in the list check whether it is equal to specific nutrients
4. If the word in the list is equal to a nutrient name, we assign the next value from the list to a dedicated nutrient column with the index of a currently considered row. The only exception are calories, in which case we assign not the next but previous value from the list to the appropriate column.

We have incorporated exception handling that omits the whole process if the first step cannot be fulfilled. This is necessary since null values can’t be split as they are considered floats.

In [15]:
for index, nutrition_string in allrecipes_df['nutrition'].iteritems():
    try:
        nutrition_list = nutrition_string.split()
        nutrition_list = [value.strip().strip(';') for value in nutrition_list]
        for i in range(len(nutrition_list)):
            
            if nutrition_list[i] == 'calories':
                allrecipes_df.loc[index, 'calories'] = nutrition_list[i-1]
                
            if nutrition_list[i] == 'protein':
                allrecipes_df.loc[index, 'protein'] = nutrition_list[i+1]
                    
            if nutrition_list[i] == 'carbohydrates':
                allrecipes_df.loc[index, 'carbohydrates'] = nutrition_list[i+1]
                
            if nutrition_list[i] == 'fat':
                allrecipes_df.loc[index, 'fat'] = nutrition_list[i+1]
                
            if nutrition_list[i] == 'cholesterol':
                allrecipes_df.loc[index, 'cholesterol'] = nutrition_list[i+1]
                
            if nutrition_list[i] == 'sodium':
                allrecipes_df.loc[index, 'sodium'] = nutrition_list[i+1]
    except:
        continue

In order to maintain data accuracy we should perform check whether data was splitter correctly. We will generate few random indexes and print both ‘nutrients’ column and individual nutrients column to compare whether data match.

In [16]:
random_index = random.sample(range(0, 1000), 5)

for index in random_index:
    print('\nRecipe number ' + str(index+1) + ':')
    print(allrecipes_df['nutrition'].iloc[index].strip('Full Nutrition'))
    print(allrecipes_df.iloc[index, 16:])


Recipe number 754:
307 calories; protein 5.4g; carbohydrates 19.3g; fat 23.6g; cholesterol 14mg; sodium 258.1mg.

calories              307
protein              5.4g
carbohydrates       19.3g
fat                 23.6g
cholesterol          14mg
sodium           258.1mg.
Name: 754, dtype: object

Recipe number 473:
46 calories; protein 0.2g; carbohydrates 5.3g; fat 2.9g; cholesterol 7.6mg; sodium 23.1mg.

calories              46
protein             0.2g
carbohydrates       5.3g
fat                 2.9g
cholesterol        7.6mg
sodium           23.1mg.
Name: 473, dtype: object

Recipe number 660:
375 calories; protein 27.1g; carbohydrates 30.4g; fat 15.5g; cholesterol 64.2mg; sodium 1250.2mg.

calories               375
protein              27.1g
carbohydrates        30.4g
fat                  15.5g
cholesterol         64.2mg
sodium           1250.2mg.
Name: 660, dtype: object

Recipe number 299:
248 calories; protein 16g; carbohydrates 25.6g; fat 9.4g; cholesterol 45.9mg; sodium 563.6m

It appears that there are no inaccuracies in the data. We can now strip trailing indicating measurement units from the end of strings in individual nutrients columns. Afterwards we can convert all the values into floats. Finally, once we are left with the numerical values only, we can include the measurement unit in the name of the column.

In [17]:
allrecipes_df['protein'] = allrecipes_df['protein'].str.rstrip('g').apply(float)
allrecipes_df['carbohydrates'] = allrecipes_df['carbohydrates'].str.rstrip('g').apply(float)
allrecipes_df['fat'] = allrecipes_df['fat'].str.rstrip('g').apply(float)
allrecipes_df['cholesterol'] = allrecipes_df['cholesterol'].str.rstrip('mg').apply(float)
allrecipes_df['sodium'] = allrecipes_df['sodium'].str.rstrip('mg.').apply(float)
allrecipes_df['calories'] = allrecipes_df['calories'].apply(float)

In [18]:
new_nutrient_names = {'protein' : 'protein [g]',
                      'carbohydrates' : 'carbohydrates [g]',
                      'fat' : 'fat [g]',
                      'cholesterol' : 'cholesterol [mg]',
                      'sodium' : 'sodium [mg]'}

allrecipes_df.rename(columns = new_nutrient_names, inplace = True)

Now that we have a column containing information on the calorie content of a recipe, we can divide it by the number of recipe servings and create a new column that would indicate calorie content per recipe portion.

In [19]:
allrecipes_df['calories_per_serving'] = np.ceil(allrecipes_df['calories'] / allrecipes_df['servings'])

We have performed some operations on the dataset by now and it is time to see how it looks at this stage.

In [20]:
allrecipes_df.head()

,name,prep,cook,additional,total,servings,yield,5 stars,4 stars,3 stars,2 stars,1 stars,nutrition,ingredients,recipe_score,number_of_reviews,calories,protein [g],carbohydrates [g],fat [g],cholesterol [mg],sodium [mg],calories_per_serving
1,Juicy Roasted Chicken,10 mins,1 hr 15 mins,15 mins,1 hr 40 mins,6,6 servings,3179.0,538.0,147.0,47.0,42.0,423 calories; protein 30.9g; carbohydrates 1.2...,"1 (3 pound) whole chicken, giblets removed,sal...",4.711358,3953.0,423.0,30.9,1.2,32.1,97.0,661.9,71.0
2,Microwave Corn on the Cob,None,5 mins,None,5 mins,1,1 serving,382.0,102.0,29.0,5.0,6.0,123 calories; protein 4.6g; carbohydrates 27.2...,"1 ear corn, husked and cleaned",4.620229,524.0,123.0,4.6,27.2,1.7,NaN,21.5,123.0
3,French Toast I,5 mins,15 mins,None,20 mins,3,6 slices french toast,1337.0,473.0,87.0,31.0,29.0,240 calories; protein 10.6g; carbohydrates 33....,"6 thick slices bread,2 eggs,⅔ cup milk,¼ teas...",4.562596,1957.0,240.0,10.6,33.6,6.4,128.3,477.7,80.0
4,The Best Banana Pudding,25 mins,None,None,25 mins,20,20 servings,830.0,117.0,30.0,15.0,15.0,329 calories; protein 4.2g; carbohydrates 56.9...,1 (5 ounce) package instant vanilla pudding mi...,4.719960,1007.0,329.0,4.2,56.9,9.6,8.6,205.2,17.0
5,Simple Macaroni and Cheese,10 mins,20 mins,None,30 mins,4,4 servings,545.0,188.0,51.0,26.0,31.0,630 calories; protein 26.5g; carbohydrates 55g...,"1 (8 ounce) box elbow macaroni,¼ cup butter,¼ ...",4.414982,841.0,630.0,26.5,55.0,33.6,99.6,777.0,158.0


Not bad. Up until now everything seems to have worked out fine, however, there is still at least one issue that we have to deal with in this data-cleaning process. You might have noticed by now that data in the time-related columns is being stored as text values. This doesn’t allow any reliable quantitative analysis. Solution to this problem would be to convert the 'prep', 'cook', 'additional' and 'total' columns’ values to unified time measurement unit - minutes.

We can achieve this by defining function convert_to_mins() and applying it to the mentioned columns, however, before it’s possible, we need to learn a little bit more about the data subset we’re working with. For that reason we are going to create a new data subset - df_time, that contains only the 'prep', 'cook', 'additional' and 'total' columns.

First we need to check what are all possible configurations of time measurement units that occur in our four columns of interest. This means checking if time is expressed in minutes, hours, days or all at once. To verify what are the possible combinations, we will begin with putting all values from the 4 columns into one list. Next, we can make sure that we are left only with string values in the list and remove all digits. Finally, we can remove all duplicate values by converting the list to set.

In [21]:
df_time = allrecipes_df.loc[:, 'prep':'total']

list_of_measures = list(df_time.loc[:, 'prep':'total'].values.T.ravel())

list_of_measures = [x for x in list_of_measures if type(x) == str]

list_of_measures = [''.join(x for x in i if not x.isdigit()) for i in list_of_measures]

set(list_of_measures)

{' day', ' days', ' hr', ' hr  mins', ' hrs', ' hrs  mins', ' min', ' mins'}

The possible combinations are - 'day', 'days', 'hr', 'hr mins', ' hrs', 'hrs mins', 'min', 'mins'. With that information we can at last define our convert_to_mins() function. The function will first split the string into list of values. Afterwards, depending on the list length and its contents (combination of time units in the list) we apply appropriate logic to convert it all to minutes.

In [22]:
def convert_to_mins(x):
    x = x.split()
    if len(x) == 4:
        x = [int(d) for d in x if d.isdigit()]
        return (x[0] * 60 + x[1])
    elif (len(x) == 2) and ('hr' in x or 'hrs' in x):
        return int(x[0]) * 60
    elif (len(x) == 2) and ('day' in x or 'days' in x):
        return int(x[0]) * 3600
    else:
        return int(x[0])

If the only time unit used is minutes, convert_to_mins() will just strip the non-digits and convert string to an integer without changing its value. In cases when days, hours or any other possible combination of time units occur, values are first converted to integers and multiplied to represent time in minutes.

We can apply our newly-created function to the df_time columns. However, since the split() method cannot be applied to NaN values (and we have those in our dataset) we will have to match the function to string values only.

In [23]:
df_time.prep = df_time.prep.map(lambda x: convert_to_mins(x) if type(x) == str else x)
df_time.cook = df_time.cook.map(lambda x: convert_to_mins(x) if type(x) == str else x)    
df_time.additional = df_time.additional.map(lambda x: convert_to_mins(x) if type(x) == str else x)
df_time.total = df_time.total.map(lambda x: convert_to_mins(x) if type(x) == str else x)

First, we need to check if our calculations were correct. It is important that we fill out all null values with zeros, which would help us in the further part of this analysis. Then we can create a new Series ‘calc_verification’ containing sum of 'prep', 'cook' and 'additional' columns. Finally, we can check whether ‘total’ column of df_time is the same as calc_verification.

In [24]:
df_time.fillna(0, inplace = True)

calc_verification = df_time.loc[:, 'prep':'additional'].sum(axis = 1, skipna = True)

df_time['total'].equals(calc_verification)

False

Turns out they are not equal. It seems that there was a mistake in our calculations. To see which fields of ‘total’ column and calc_verification Series are not matching, we will create a bumpy array  ‘diff’. It will contain series of 1s and 0s representing whether values match or not. Afterwards, we can subset the df_time with diff to see the problematic records.

In [25]:
diff = np.where( df_time['total'] == calc_verification , 1, 0)

df_time[diff == 0]

,prep,cook,additional,total
159,20.0,10.0,3600.0,3600.0
288,20.0,10.0,3600.0,3600.0
818,10.0,15.0,10800.0,10800.0
844,5.0,0.0,7200.0,7200.0


So the problematic records are in rows 159, 288, 818 and 844. It seems that whenever days were used as a measurement of time, the 'prep' and 'cook' were not added in the 'total' column's value. It looks like we haven't made a mistake after all but fallen victim of certain data simplification at the source. We could leave those data inconsistencies in the dataset but since those are only 4 records, we can as well update the values manually.

In [26]:
df_time.at[159, 'total'] = 3630.00
df_time.at[288, 'total'] = 3630.00
df_time.at[818, 'total'] = 10825.00
df_time.at[844, 'total'] = 7205.00

Let's once again check whether all the values in df_time[‘total’] and ‘calc_verification’ are the same.

In [27]:
df_time['total'].equals(calc_verification)

True

Looks like it worked out right. Now we can substitute time-related data in the original dataframe - allrecipes_df with data from df_time. However, beforehand, we should convert all 0 back to null values to avoid any future inaccuracies.

In [28]:
df_time.replace(0, np.nan, inplace=True)

allrecipes_df.loc[:, 'prep':'total'] = df_time

We have successfully converted time-related data to numerical values. Let’s have a look at our data right now.

In [29]:
allrecipes_df.head()

,name,prep,cook,additional,total,servings,yield,5 stars,4 stars,3 stars,2 stars,1 stars,nutrition,ingredients,recipe_score,number_of_reviews,calories,protein [g],carbohydrates [g],fat [g],cholesterol [mg],sodium [mg],calories_per_serving
1,Juicy Roasted Chicken,10.0,75.0,15.0,100.0,6,6 servings,3179.0,538.0,147.0,47.0,42.0,423 calories; protein 30.9g; carbohydrates 1.2...,"1 (3 pound) whole chicken, giblets removed,sal...",4.711358,3953.0,423.0,30.9,1.2,32.1,97.0,661.9,71.0
2,Microwave Corn on the Cob,NaN,5.0,NaN,5.0,1,1 serving,382.0,102.0,29.0,5.0,6.0,123 calories; protein 4.6g; carbohydrates 27.2...,"1 ear corn, husked and cleaned",4.620229,524.0,123.0,4.6,27.2,1.7,NaN,21.5,123.0
3,French Toast I,5.0,15.0,NaN,20.0,3,6 slices french toast,1337.0,473.0,87.0,31.0,29.0,240 calories; protein 10.6g; carbohydrates 33....,"6 thick slices bread,2 eggs,⅔ cup milk,¼ teas...",4.562596,1957.0,240.0,10.6,33.6,6.4,128.3,477.7,80.0
4,The Best Banana Pudding,25.0,NaN,NaN,25.0,20,20 servings,830.0,117.0,30.0,15.0,15.0,329 calories; protein 4.2g; carbohydrates 56.9...,1 (5 ounce) package instant vanilla pudding mi...,4.719960,1007.0,329.0,4.2,56.9,9.6,8.6,205.2,17.0
5,Simple Macaroni and Cheese,10.0,20.0,NaN,30.0,4,4 servings,545.0,188.0,51.0,26.0,31.0,630 calories; protein 26.5g; carbohydrates 55g...,"1 (8 ounce) box elbow macaroni,¼ cup butter,¼ ...",4.414982,841.0,630.0,26.5,55.0,33.6,99.6,777.0,158.0


Up until this point we have been working solely on data collected from allrecipes.com. There is no doubt that there are a lot of insights to be gathered from that data alone. However, we can enrich this dataset even further.

I’ve decided to compile a list of herbs from Encyclopedia Britannica, which is available in the herbs_britannica.txt file within the directory. In the next few steps we will create a number of columns representing whether a herb appear in a recipe or not. 

The steps we will follow could be used to supplement dataset with information not only about herbs, but virtually all other categories of ingredients. In this case, however, we will limit the scope of newly added data for it is only to show a certain proof of concept.

We will begin with importing the list of herbs from herbs_britannica.txt and performing some basic string formatting to clean the data.

In [30]:
with open('herbs_britannica.txt', 'r') as file:
    file_content = file.read()
    herbs = file_content.split('\n')
    herbs = [item.lower().strip() for item in herbs]
    herbs = list(set(herbs))
    herbs.remove('')

The outcome of that operation is a list of well-formatted herbs’ names. Next step is to create a create_ingredient_column() function that we can supply with a herb’s name and it creates a new column in allrecipes_df representing with 1s and 0s whether recipe contain a particular herb.

In [74]:
def create_ingredient_column(ingredient_name):
    ingredient = str(ingredient_name)
    allrecipes_df[ingredient] = np.where(allrecipes_df['ingredients'].str.find(ingredient) != -1, 1, 0)

Now that we have defined our function, we can iterate over the list of herbs and supply create_ingredient_column() function with herbs’ names one by one.

In [42]:
for herb in herbs:
    create_ingredient_column(herb)

At this point it would be nice to check which herbs didn’t occur in any of the recipes. To check how many herbs like that we have, we can count the number of columns in which all values equals to 0. We will also check how many columns with 0s and 1s exist in our dataset as well. We will show how this issue can be approached in two different ways. First is more of a custom one and the second utilises pandas capabilities to higher extend.

In [33]:
count_of_herbs_columns = {}

for column in allrecipes_df.columns:
    if column in herbs:
        value_counts = np.array2string(allrecipes_df[column].value_counts().index.values)
        if value_counts not in count_of_herbs_columns.keys():
            count_of_herbs_columns[value_counts] = 1
        else:
            count_of_herbs_columns[value_counts] += 1

print (count_of_herbs_columns)

{'[0]': 39, '[0 1]': 45, '[1 0]': 1}


And here is maybe a bit more intuitive way to achieve similar result.

In [48]:
allrecipes_df.loc[:, 'cinnamon':'wasabi'].sum().value_counts()

0      39
1       8
3       5
2       4
5       4
68      2
7       2
36      1
6       1
16      1
19      1
23      1
26      1
28      1
509     1
124     1
41      1
45      1
182     1
61      1
62      1
66      1
325     1
73      1
346     1
69      1
54      1
37      1
dtype: int64

There are 39 herbs which doesn't exist in any recipe and none that would occur in all of them. We can then assume that 46 herb-related columns can be of any use to us. Considering that, in the next step we will remove all herb-related columns where all values equals to 0 from the dataset.

In [53]:
nunique = allrecipes_df.nunique()
columns_to_drop = nunique[nunique == 1].index
allrecipes_df = allrecipes_df.drop(columns_to_drop, axis=1)

Let’s check to what extend have remaining herbs been used in our recipes.

In [72]:
ingr_sum_dict = {}

for column in allrecipes_df.columns:
    if column in herbs:
        ingr_sum_dict[column] = allrecipes_df[column].sum()

print(ingr_sum_dict)
print('\n', sorted(ingr_sum_dict.values()))

{'cinnamon': 66, 'celery': 61, 'pepper': 509, 'garlic': 346, 'cilantro': 36, 'black pepper': 325, 'parsley': 68, 'vanilla': 124, 'sage': 26, 'rosemary': 23, 'oregano': 68, 'thyme': 41, 'cayenne pepper': 69, 'sesame': 28, 'clove': 182, 'cumin': 45, 'ginger': 37, 'basil': 54, 'paprika': 73, 'mustard': 62, 'anise': 3, 'coriander': 3, 'mint': 3, 'nutmeg': 16, 'chili pepper': 1, 'celery seed': 6, 'turmeric': 2, 'curry': 7, 'star anise': 1, 'dill': 19, 'savory': 1, 'bay leaf': 7, 'allspice': 3, 'chives': 5, 'plantain': 1, 'horseradish': 2, 'marjoram': 5, 'dandelion': 1, 'poppy seed': 5, 'mace': 1, 'tarragon': 5, 'fennel': 2, 'brown mustard': 2, 'lemon grass': 1, 'cardamom': 3, 'wasabi': 1}

 [1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 5, 5, 5, 5, 6, 7, 7, 16, 19, 23, 26, 28, 36, 37, 41, 45, 54, 61, 62, 66, 68, 68, 69, 73, 124, 182, 325, 346, 509]


We can set a threshold of let’s say 20 and check which herbs have been used at least that many times.

In [73]:
filtered_ingr_sum_dict =  {key: value for (key, value) in ingr_sum_dict.items() if value >= 20}
print(filtered_ingr_sum_dict)

{'cinnamon': 66, 'celery': 61, 'pepper': 509, 'garlic': 346, 'cilantro': 36, 'black pepper': 325, 'parsley': 68, 'vanilla': 124, 'sage': 26, 'rosemary': 23, 'oregano': 68, 'thyme': 41, 'cayenne pepper': 69, 'sesame': 28, 'clove': 182, 'cumin': 45, 'ginger': 37, 'basil': 54, 'paprika': 73, 'mustard': 62}


There are quite a few left. If a herb hasn’t occurred in at least 20 recipes (this threshold is arbitrary) it will be of a limited value to us in the further analysis. This means that we can remove the irrelevant columns from the dataset. 

In [37]:
for column in allrecipes_df.columns:    
    if (column in herbs) and (column not in list(filtered_ingr_sum_dict.keys())):
        allrecipes_df.drop(labels = column, axis = 1, inplace = True)

Across this whole process of preparing data, we have fixed formatting, splitter and merged data, converted data types and created sets of totally new data points. We can now print the final result of this all.

In [38]:
allrecipes_df.head()

,name,prep,cook,additional,total,servings,yield,5 stars,4 stars,3 stars,2 stars,1 stars,nutrition,ingredients,recipe_score,number_of_reviews,calories,protein [g],carbohydrates [g],fat [g],cholesterol [mg],sodium [mg],calories_per_serving,cinnamon,celery,pepper,garlic,cilantro,black pepper,parsley,vanilla,sage,rosemary,oregano,thyme,cayenne pepper,sesame,clove,cumin,ginger,basil,paprika,mustard
1,Juicy Roasted Chicken,10.0,75.0,15.0,100.0,6,6 servings,3179.0,538.0,147.0,47.0,42.0,423 calories; protein 30.9g; carbohydrates 1.2...,"1 (3 pound) whole chicken, giblets removed,sal...",4.711358,3953.0,423.0,30.9,1.2,32.1,97.0,661.9,71.0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Microwave Corn on the Cob,NaN,5.0,NaN,5.0,1,1 serving,382.0,102.0,29.0,5.0,6.0,123 calories; protein 4.6g; carbohydrates 27.2...,"1 ear corn, husked and cleaned",4.620229,524.0,123.0,4.6,27.2,1.7,NaN,21.5,123.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,French Toast I,5.0,15.0,NaN,20.0,3,6 slices french toast,1337.0,473.0,87.0,31.0,29.0,240 calories; protein 10.6g; carbohydrates 33....,"6 thick slices bread,2 eggs,⅔ cup milk,¼ teas...",4.562596,1957.0,240.0,10.6,33.6,6.4,128.3,477.7,80.0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,The Best Banana Pudding,25.0,NaN,NaN,25.0,20,20 servings,830.0,117.0,30.0,15.0,15.0,329 calories; protein 4.2g; carbohydrates 56.9...,1 (5 ounce) package instant vanilla pudding mi...,4.719960,1007.0,329.0,4.2,56.9,9.6,8.6,205.2,17.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,Simple Macaroni and Cheese,10.0,20.0,NaN,30.0,4,4 servings,545.0,188.0,51.0,26.0,31.0,630 calories; protein 26.5g; carbohydrates 55g...,"1 (8 ounce) box elbow macaroni,¼ cup butter,¼ ...",4.414982,841.0,630.0,26.5,55.0,33.6,99.6,777.0,158.0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Amazing! We are left with a great dataset prepared for further analysis. At this point we can save it for the use in the future.

In [40]:
allrecipes_df.to_csv('allrecipes_data.csv')